In [ ]:
## This script calculates the majority vote simply using all the submissions
## Input: MTurk submission CSV
## Output: pixel-wise human reference of the test images

In [ ]:
# step 0. load packages
import cv2, os
import numpy as np
import pandas as pd
from matplotlib.path import Path
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
# step 1. load submission csv
SPECIES = 'palm' ## 'palm' or 'cec'
dat_sub_full = pd.read_csv('../data/MTurk/MTurk_' + SPECIES + '.csv')
## filter out rejected submissions
dat_sub = dat_sub_full[dat_sub_full['AssignmentStatus'] != 'Rejected']
dat_sub.index = range(dat_sub.shape[0])

In [ ]:
# step 2. function definition
## function for transforming polygon to mask
def poly2mask(nx, ny, poly_verts):
    # Create vertex coordinates for each grid cell...
    # (<0,0> is at the top left of the grid in this system)
    x, y = np.meshgrid(np.arange(nx), np.arange(ny))
    x, y = x.flatten(), y.flatten()
    points = np.vstack((x,y)).T
    path = Path(poly_verts)
    grid = path.contains_points(points)
    grid = grid.reshape((ny,nx))
    return grid

In [ ]:
# step 3. loop over the raw images
main_folder = '../data/shadow_free/'
img_list = os.listdir(main_folder)
img_list.sort()
for idx in range(len(img_list)):
    ## load the image
    time_start = datetime.now()
    print(img_list[idx])
    img = cv2.imread(main_folder + img_list[idx])
    ## initialize dens array
    dens_arr = np.zeros([10000, 10000]) - 1
    ## loop over patches
    for i in range(10):
        for j in range(10):
            idx_img = idx * 100 + i * 10 + j + 1
            url_img = 'https://raw.githubusercontent.com/ChengliangTang/rainforest_images/master/patches_free/img_' \
            + '{0:04}'.format(idx_img) + '.jpg'
            loc = np.where(dat_sub['Input.img_url'] == url_img)[0]
            num_patch = len(loc)
            dens_mat = np.zeros([1000, 1000, num_patch])
            idx_patch = 0
            for id_sub in loc:
                sub_mat = np.zeros([1000, 1000])
                polyString = dat_sub['Answer.coordinates'][id_sub]
                polyString = polyString.replace('[', '')
                polyString = polyString.replace(']', '')
                polyString = polyString.replace('"x":', '')
                polyString = polyString.replace(',"y":', ';')
                polyString = polyString.replace(',', '')
                polyList = polyString.split('{-1;-1}')
                polyList.pop()
                for polyGon in polyList:
                    elementList = polyGon.split('}{')
                    ## transform element list into integers
                    for k in range(len(elementList)):
                        element = elementList[k].replace('{', '')
                        element = element.replace('}', '')
                        (x, y) = element.split(';')
                        x = int(float(x)) - 1
                        y = int(float(y)) - 1
                        elementList[k] = (max([0, x]), max([0, y]))
                    polyMask = poly2mask(1000, 1000, elementList)
                    sub_mat[polyMask == True] = 1
                dens_mat[:, :, idx_patch] = sub_mat
                idx_patch += 1
            dens_mat = np.sum(dens_mat, axis=2)
            dens_mat[dens_mat < 5] = 0
            dens_mat[dens_mat > 0] = 1
            r1 = i * 1000
            r2 = r1 + 1000
            c1 = j * 1000
            c2 = c1 + 1000
            dens_arr[r1:r2, c1:c2] = dens_mat
    ## create visualizations
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_new = np.zeros_like(img)
    img_new[:,:,0] = img_gray
    img_new[:,:,1] = img_gray
    img_new[:,:,2] = img_gray
    img_new[dens_arr == 1] = img[dens_arr == 1]
    img_new[dens_arr == -1] = 0
    cv2.imwrite('../output/mref/' + SPECIES + '/' + img_list[idx], img_new)
    pd.DataFrame(dens_arr).to_csv('../output/mref/' + SPECIES + '/' + img_list[idx][:(-4)] + '.csv')
    ## print process time
    print(datetime.now() - time_start)
print('Task finished!')